## Генератор лога ошибок для регионов
1. В папку `source` поместить файлы ошибок записи к врачу, скачанные с файлообменника, в формате `.csv` (т.е. как они есть);
2. `reg_ref.csv` - спрвочник названий регионов, он должен быть в папке скрипта;
3. Запсустить скрипт: будет сформирован архив с логами ошибок по регионам за период. Период автоматически определится на основании содержания файлов ошибок записи.
Формат названия сформированного архива: `error_log_<дата_начала_периода>_<дата_окончания_периода>_<дата_формирования_архива>_<время_формирования_архива>.zip`;
4. Profit!

In [1]:
%%time

import pandas as pd
from datetime import date
import datetime
import os
import shutil # упаковка в архив
import warnings # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

# пути к файлам
dir_path = os.getcwd().replace("\\", "/")
source_path = dir_path + '/source'
save_path = dir_path + '/error_log/'
file_list = os.listdir(source_path)

# очистка папки для логов ошибок (если она есть)
try:
    shutil.rmtree(save_path)
except Exception:
    pass
# создание пустой папки для логов ошибок
try:
    os.mkdir(save_path)
except Exception:
    pass


# словарь импортных названий регионов
df_reg_ref = pd.read_csv(dir_path + '/reg_ref.csv', sep=';')
reg_dict = {}
for index, row in df_reg_ref.iterrows():
    reg_dict[row.reg_name_rus] = row.reg_name_eng

df_errors = pd.DataFrame()
for _file in file_list:
    path_day = source_path + '/' + _file
    try:
        df_day = pd.read_csv(path_day, sep=";")
    except Exception:
        print(f'Что-то не то с файлом {path_day}')
    print(f'число строк в файле {path_day}: {df_day.shape[0]}')
    df_errors = pd.concat([df_errors, df_day])
    print(f'число строк в итоговом файле: {df_errors.shape[0]}')
df_errors = df_errors.reset_index(drop=True).rename(columns={df_errors.columns[1]: "region_name", df_errors.columns[11]: "appointment_type"})
df_errors["create_ts"] = pd.to_datetime(df_errors["create_ts"], format='ISO8601')
# диапазон дат для названий файлов
range_name = '_' + str(min(df_errors.create_ts)).split(' ')[0] + '_' + str(max(df_errors.create_ts)).split(' ')[0]
df_errors_3 = df_errors[df_errors.appointment_type == 3].reset_index(drop=True)
df_errors_3['region_name'] = df_errors_3['region_name'].fillna('пустой регион')
local_list = list(set(df_errors_3.region_name.tolist()))
for locality in local_list:
    df_local = df_errors_3[(df_errors_3.region_name == locality) & (df_errors_3.final_state != 'Ошибка отсутсвует')]
    df_local = df_local[['session_id', 'create_ts','final_state', 'step_name', 'mo_name', 'Post_name']]
    print(f'регион был: {locality}')
    try:
        locality = reg_dict[locality]
    except Exception:
        print(f'В словаре нет региона {locality}')
        # locality = 'nan_region'
    print(f'регион стал: {locality}')
    save_path_local = save_path + locality + '_errors' + range_name + '.csv'
    df_local.to_csv(save_path_local, sep=";", encoding="utf-8-sig", index=False)
    print(f'сохранён файл {save_path_local}')
# архивация
# текущее время
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")


dir_name = "error_log" + range_name + '_' + str(date.today()) + '_' + current_time.replace(':', '-')
zip_path = shutil.make_archive(dir_name, 'zip', save_path)
print(f'Архив {dir_name}')
# очистка папки для логов ошибок
try:
    shutil.rmtree(save_path)
except Exception:
    pass
print('Happy End!')

число строк в файле c:/work/error_log/source/01.11.csv: 524928
число строк в итоговом файле: 524928
число строк в файле c:/work/error_log/source/02.11.csv: 496113
число строк в итоговом файле: 1021041
число строк в файле c:/work/error_log/source/03.11.csv: 401161
число строк в итоговом файле: 1422202
число строк в файле c:/work/error_log/source/04.11.csv: 190725
число строк в итоговом файле: 1612927
число строк в файле c:/work/error_log/source/05.11.csv: 210879
число строк в итоговом файле: 1823806
число строк в файле c:/work/error_log/source/30.10.csv: 590943
число строк в итоговом файле: 2414749
число строк в файле c:/work/error_log/source/31.10.csv: 534871
число строк в итоговом файле: 2949620
регион был: Орловская область
регион стал: OrlovskayaOblast
сохранён файл c:/work/error_log/error_log/OrlovskayaOblast_errors_2023-10-30_2023-11-05.csv
регион был: Чеченская Республика
регион стал: ChechenskayaResp
сохранён файл c:/work/error_log/error_log/ChechenskayaResp_errors_2023-10-30_20